In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

In [2]:
data = pd.read_csv('train.csv')
data.head()

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09


In [3]:
data.shape

(7340, 18)

In [4]:
cal = pd.read_csv('train_calendar.csv')
cal.head()

,date,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,warehouse
0,2022-03-09,NaN,0,0,0,0,0,1,0,0,0.0,0,0.0,0.0,Prague_1
1,2019-07-03,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
2,2020-05-18,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
3,2021-11-09,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
4,2023-08-02,NaN,0,0,0,0,0,0,0,0,0.0,0,2.5,0.0,Prague_1


In [5]:
cal.shape

(13307, 15)

In [6]:
data.isnull().sum()

warehouse                    0
date                         0
orders                       0
holiday_name              7122
holiday                      0
shutdown                     0
mini_shutdown                0
shops_closed                 0
winter_school_holidays       0
school_holidays              0
blackout                     0
mov_change                   0
frankfurt_shutdown           0
precipitation              270
snow                       270
user_activity_1              0
user_activity_2              0
id                           0
dtype: int64

In [7]:
cal.isnull().sum()

date                          0
holiday_name              12744
holiday                       0
shutdown                      0
mini_shutdown                 0
warehouse_limited             0
shops_closed                  0
winter_school_holidays        0
school_holidays               0
blackout                      0
mov_change                    0
frankfurt_shutdown            0
precipitation              2108
snow                       2108
warehouse                     0
dtype: int64

In [8]:
data.describe()

,orders,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2
count,7340.000000,7340.000000,7340.000000,7340.000000,7340.000000,7340.000000,7340.000000,7340.000000,7340.000000,7340.000000,7070.000000,7070.000000,7340.000000,7340.000000
mean,5535.443869,0.027248,0.000136,0.000545,0.010899,0.029973,0.007084,0.000954,0.221499,0.000272,1.621126,0.290112,1633.545777,23477.031335
std,2182.680279,0.162816,0.011672,0.023340,0.103836,0.170524,0.083876,0.030869,0.410293,0.016506,4.542875,1.546102,784.167637,8934.848328
min,790.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,482.000000,1797.000000
25%,4434.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,945.000000,18509.000000
50%,5370.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1349.000000,22267.000000
75%,7009.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2308.000000,30028.750000
max,18139.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,80.000000,19.840000,3437.000000,45769.000000


In [9]:
def fill_missing_vals(df):
    numerical_cols = df.select_dtypes(include=['float64', 'int64'])
    catergorical_cols = df.select_dtypes(include= ['object'])
    
    catergorical_cols['holiday_name'].fillna('workday' , inplace = True)
    imputer = KNNImputer(n_neighbors=5)
    imputed_data = imputer.fit_transform(numerical_cols)
    
    imputed_df = pd.DataFrame(data =imputed_data , columns=numerical_cols.columns )
    return imputed_df , catergorical_cols

In [10]:
num , cat  = fill_missing_vals(data)
num1 , cat1 = fill_missing_vals(cal)

In [11]:
df = pd.concat([cat , num] , axis = 'columns')
df1 = pd.concat([cat1 , num1] , axis = 'columns')

In [12]:
# df['date'] = pd.to_datetime(df['date'] , format='%Y-%m-%d')
# df1['date'] = pd.to_datetime(df['date'] , format='%Y-%m-%d')

## Exploratory Data Analysis

In [13]:
df1.head()

,date,holiday_name,warehouse,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow
0,2022-03-09,workday,Prague_1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-07-03,workday,Prague_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-05-18,workday,Prague_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-11-09,workday,Prague_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-08-02,workday,Prague_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0


In [14]:
df1['id'] = df1['warehouse'] + '_' + df1['date'].astype(str)

In [15]:
df['date'] = pd.to_datetime(df['date'] , format='%Y-%m-%d' )
df1['date'] = pd.to_datetime(df['date'] , format='%Y-%m-%d')

In [16]:
df = df.drop_duplicates(keep='first')
df1 = df1.drop_duplicates(keep='first')

In [17]:
df.shape , df1.shape

((7340, 18), (13307, 16))

In [18]:
missing_combinations = df1[~df1['id'].isin(df['id'])]
missing_combinations.shape

(5967, 16)

In [19]:
missing_combinations.sort_values(by = 'date').sort_values(by='date')

,date,holiday_name,warehouse,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,id
1,2020-12-06,workday,Prague_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Prague_1_2019-07-03
3580,2020-12-06,workday,Brno_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brno_1_2019-01-21
2,2020-12-07,workday,Prague_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Prague_1_2020-05-18
3581,2020-12-07,workday,Brno_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Brno_1_2019-08-08
3583,2020-12-09,workday,Brno_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,Brno_1_2019-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13300,NaT,workday,Budapest_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.1,0.0,Budapest_1_2020-02-05
13301,NaT,workday,Budapest_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.1,0.0,Budapest_1_2019-10-05
13302,NaT,workday,Budapest_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.2,0.0,Budapest_1_2019-09-09
13303,NaT,workday,Budapest_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.7,0.0,Budapest_1_2020-10-26


In [20]:
missing_combinations.dropna(inplace=True)

C:\Users\windows\AppData\Local\Temp\ipykernel_21924\4192529783.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_combinations.dropna(inplace=True)


In [24]:
new_df = missing_combinations[[ 'date' , 'warehouse']].copy()

In [25]:
for col in missing_combinations.columns:
    new_df[col] = missing_combinations[col]

In [26]:
new_df

,date,warehouse,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,id
1,2020-12-06,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2019-07-03
2,2020-12-07,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2020-05-18
12,2020-12-17,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2020-06-11
13,2020-12-18,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2020-10-11
14,2020-12-19,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2019-03-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7332,2024-03-07,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2019-10-30
7333,2024-03-08,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2019-01-23
7334,2024-03-09,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2020-04-05
7336,2024-03-11,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2020-04-23


In [29]:
df_columns = set(df.columns) - set(df1.columns) - {'id'}
for col in df_columns:
    new_df[col] = pd.NA

In [30]:
new_df

,date,warehouse,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,id,user_activity_2,user_activity_1,orders
1,2020-12-06,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2019-07-03,NaN,NaN,NaN
2,2020-12-07,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2020-05-18,NaN,NaN,NaN
12,2020-12-17,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2020-06-11,NaN,NaN,NaN
13,2020-12-18,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2020-10-11,NaN,NaN,NaN
14,2020-12-19,Prague_1,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,Prague_1_2019-03-13,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7332,2024-03-07,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2019-10-30,NaN,NaN,NaN
7333,2024-03-08,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2019-01-23,NaN,NaN,NaN
7334,2024-03-09,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2020-04-05,NaN,NaN,NaN
7336,2024-03-11,Prague_3,workday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502,0.156,Prague_3_2020-04-23,NaN,NaN,NaN


In [34]:
combine_df = pd.concat([df , new_df] , ignore_index=True)
combine_df.shape

(10082, 19)

In [42]:
combine_df = combine_df.drop_duplicates()
combine_df['orders'] = combine_df['orders'].fillna(0)
combine_df = combine_df.drop(columns=['holiday_name' , 'warehouse_limited' , 'id'] )

In [43]:
combine_df.head()

,warehouse,date,orders,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2
0,Prague_1,2020-12-05,6895.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1722.0,32575.0
1,Prague_1,2020-12-06,6584.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1688.0,32507.0
2,Prague_1,2020-12-07,7030.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1696.0,32552.0
3,Prague_1,2020-12-08,6550.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,1681.0,32423.0
4,Prague_1,2020-12-09,6910.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,1704.0,32410.0


In [48]:
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(combine_df[['user_activity_1' , 'user_activity_2']])
combine_df[['user_activity_1' , 'user_activity_2']] = imputed_data

In [49]:
combine_df.isnull().sum()

warehouse                 0
date                      0
orders                    0
holiday                   0
shutdown                  0
mini_shutdown             0
shops_closed              0
winter_school_holidays    0
school_holidays           0
blackout                  0
mov_change                0
frankfurt_shutdown        0
precipitation             0
snow                      0
user_activity_1           0
user_activity_2           0
dtype: int64